In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
import time
from datetime import datetime
import math
from IPython.display import display, Audio
from scipy.io import wavfile
from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

C:\Users\zankyo\Anaconda3\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\zankyo\Anaconda3\lib\site-packages\torchaudio\backend\utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 to '


# Parameter

In [13]:
label_dir = "C:/Users/zankyo/Desktop/yamamoto/datasets/cleandata/"
train_dir = "C:/Users/zankyo/Desktop/yamamoto/datasets/noisedata/"

dnn_dir="C:/Users/zankyo/Desktop/yamamoto/ERNN/ERNN/dnn_dir/"
if(os.path.isdir(dnn_dir)==False):
    os.mkdir(dnn_dir)

In [2]:
# stft
winLen = 512
shiftLen = 256
fftLen = winLen # ch Num
fftDecimate = 1
chNum = int(winLen/fftDecimate/2+1)

# save dnn directory
dnn_dir  = './dnn_dir/'
if(os.path.isdir(dnn_dir)==False):
    os.mkdir(dnn_dir)
    
# train parameter
loadSize = 2048
batchSize = 16 
Log_reg = 10**(-6)
batchTimeFrame = 68

MAX_EPOCH         = 200
# Adam setting
lr           = 0.0001

# GPU setup

`gpu_data = cpu_data.to(device)`

In [3]:
print("GPU is available:",torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GPU is available: True


# Load datasets

In [69]:
print("...Loading...")



class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, transform = None):
        self.transform = transform

        self.data = datalist
        self.label = labellist

        self.datanum = len(self.data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label
    
dataset = Mydatasets()
print(len(dataset))

trainset ,testset = train_test_split(dataset,test_size = 0.2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batchSize, shuffle = True, num_workers = 2)
testloader = torch.utils.data.DataLoader(testset, batch_size = batchSize, shuffle = True, num_workers = 2)

print("train-data size:", len(trainset),"/",len(dataset))
print("test-data size:",len(testset),"/",len(dataset))

...Loading...
15603
train-data size: 12482 / 15603
test-data size: 3121 / 15603


# Neural Network

In [76]:
# ERNN
class ERNN_K1(nn.Module):
    def __init__(self, inputSize, hiddenSize, stateSize):
        super(ERNN_K1, self).__init__()
        with self.init_scope():
            self.stateSize = stateSize
            self.l1_in = nn.Linear(inputSize, stateSize)
            self.l1_hid = nn.Linear(stateSize, stateSize)
            self.l2 = nn.Linear(stateSize, hiddenSize)
            self.l3 = nn.Linear(hiddenSize, stateSize)
            self.l_out = nn.Linear(stateSize, inputSize)
            
            self.initEta = 1e-2
            self.eta1 = variable.Parameter(self.initEta, 1)
            
             
    def __call__(self, x0):
        Bsize,Fsize,Tsize = x0.shape
        h0 = Variable( xp.zeros((Bsize,self.stateSize)).astype(np.float32) )
        y = xp.zeros((Bsize,Fsize,0)).astype(np.float32)
        for ii in range(Tsize):
            wx = self.l1_in(x0[:,:,ii])
            
            #K=1
            uh0 = self.l1_hid(h0+h0)
            F1h_hat = F.relu(uh0 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h1 = h0 + self.eta1 * (Fnh_hat - (h0+h0))
            
            y1 = F.sigmoid(self.l_out(h1))
            y1 = F.expand_dims(y1, axis=2)
            y = F.concat( (y,y1), axis=2)
            h0 = copy.deepcopy(h1)
        return y
    
################################################################################
class ERNN_K3(nn.Module):
    def __init__(self, inputSize, hiddenSize, stateSize):
        super(ERNN_K3, self).__init__()
        with self.init_scope():
            self.stateSize = stateSize
            self.l1_in = L.Linear(inputSize, stateSize)
            self.l1_hid = L.Linear(stateSize, stateSize)
            self.l2 = L.Linear(stateSize, hiddenSize)
            self.l3 = L.Linear(hiddenSize, stateSize)
            self.l_out = L.Linear(stateSize, inputSize)
            
            
            self.initEta = 1e-2
            self.eta1 = variable.Parameter(self.initEta, 1)
            self.eta2 = variable.Parameter(self.initEta, 1)
            self.eta3 = variable.Parameter(self.initEta, 1)
            
             
    def __call__(self, x0):
        Bsize,Fsize,Tsize = x0.shape
        h0 = Variable( xp.zeros((Bsize,self.stateSize)).astype(np.float32) )
        y = xp.zeros((Bsize,Fsize,0)).astype(np.float32)
        for ii in range(Tsize):
            wx = self.l1_in(x0[:,:,ii])
            
            #K=1
            uh0 = self.l1_hid(h0+h0)
            F1h_hat = F.relu(uh0 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h1 = h0 + self.eta1 * (Fnh_hat - (h0+h0))
            
            #K=2
            uh1 = self.l1_hid(h1+h0)
            F1h_hat = F.relu(uh1 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h2 = h1 + self.eta2 * (Fnh_hat - (h1+h0))
            
            #K=3
            uh2 = self.l1_hid(h2+h0)
            F1h_hat = F.relu(uh2 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h3 = h2 + self.eta3 * (Fnh_hat - (h2+h0))
            
            y1 = F.sigmoid(self.l_out(h3))
            y1 = F.expand_dims(y1, axis=2)
            y = F.concat( (y,y1), axis=2)
            h0 = copy.deepcopy(h3)
        return y
################################################################################
class ERNN_K5(nn.Module):
    def __init__(self, inputSize, hiddenSize, stateSize):
        super(ERNN_K5, self).__init__()
        with self.init_scope():
            self.stateSize = stateSize
            self.l1_in = L.Linear(inputSize, stateSize)
            self.l1_hid = L.Linear(stateSize, stateSize)
            self.l2 = L.Linear(stateSize, hiddenSize)
            self.l3 = L.Linear(hiddenSize, stateSize)
            self.l_out = L.Linear(stateSize, inputSize)
            
            
            self.initEta = 1e-2
            self.eta1 = variable.Parameter(self.initEta, 1)
            self.eta2 = variable.Parameter(self.initEta, 1)
            self.eta3 = variable.Parameter(self.initEta, 1)
            self.eta4 = variable.Parameter(self.initEta, 1)
            self.eta5 = variable.Parameter(self.initEta, 1)
            
             
    def __call__(self, x0):
        Bsize,Fsize,Tsize = x0.shape
        h0 = Variable( xp.zeros((Bsize,self.stateSize)).astype(np.float32) )
        y = xp.zeros((Bsize,Fsize,0)).astype(np.float32)
        for ii in range(Tsize):
            wx = self.l1_in(x0[:,:,ii])
            
            #K=1
            uh0 = self.l1_hid(h0+h0)
            F1h_hat = F.relu(uh0 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h1 = h0 + self.eta1 * (Fnh_hat - (h0+h0))
            
            #K=2
            uh1 = self.l1_hid(h1+h0)
            F1h_hat = F.relu(uh1 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h2 = h1 + self.eta2 * (Fnh_hat - (h1+h0))
            
            #K=3
            uh2 = self.l1_hid(h2+h0)
            F1h_hat = F.relu(uh2 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h3 = h2 + self.eta3 * (Fnh_hat - (h2+h0))
            
            #K=4
            uh3 = self.l1_hid(h3+h0)
            F1h_hat = F.relu(uh3 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h4 = h3 + self.eta4 * (Fnh_hat - (h3+h0))
            
            #K=5
            uh4 = self.l1_hid(h4+h0)
            F1h_hat = F.relu(uh4 + wx)
            Fnh_hat = F.relu(self.l3(F.relu(self.l2(F1h_hat)))) # kokoni DNN wo ippai irerareru
            h5 = h4 + self.eta5 * (Fnh_hat - (h4+h0))
            
            y1 = F.sigmoid(self.l_out(h5))
            y1 = F.expand_dims(y1, axis=2)
            y = F.concat( (y,y1), axis=2)
            h0 = copy.deepcopy(h3)
        return y   
    

# ---------------------------------------------

# modules.py

In [ ]:
#import modules
import numpy as np
import scipy as scipy
from scipy.io import wavfile
# from chainer import cuda
import glob
import sys
import math
# cuda.check_cuda_available()
# xp = cuda.cupy 


#####################################################################
# transfer to GPU 
def list_to_gpu(ll):
    rr = []
    for ii in range(len(ll)):
        rr.append( cuda.to_gpu( ll[ii] ) )
    return rr

#####################################################################
# read & write audio file
def wavread(fn):
    fs, data = wavfile.read(fn)
    data     = (data.astype(np.float32) / 2**(15))
    return data, fs

def wavwrite(fn, data, fs):
    data = scipy.array(scipy.around(data * 2**(15)), dtype = "int32")
    wavfile.write(fn, fs, data)

def loadDataset(clean_dir, noisy_dir, speech_per_set, testFlag):
    c_files = glob.glob(clean_dir + "/" + "*.wav")
    n_files = glob.glob(noisy_dir + "/" + "*.wav")
    Num_wav   = len( c_files )
    if testFlag == 1:
        Num_wav = round(Num_wav*0.01)
    perm      = np.random.permutation( Num_wav )

    S_all = []
    S_set = []
    cnt   = 0
    print( 'Loading... (Training set)' )
    for ii in range( Num_wav ):
        if( ii%int(Num_wav/20) == 0 ):
            sys.stdout.write( '\r   '+str(ii+1)+'/'+str( Num_wav ) )
            sys.stdout.flush()
        c_fn = c_files[perm[ii]]
        n_fn = n_files[perm[ii]]
        s, org_fs = wavread( c_fn )
        x, org_fs = wavread( n_fn ) 
        S_set.append([s,x])
        cnt += 1
        if(cnt == speech_per_set):
            cnt = 0
            S_all.append( S_set )
            S_set = []
    sys.stdout.write('\n')
    S_all.append( S_set )
    return S_all

########################################################################################
# DGT config
def hannWin(N):
    window = 0.5 + 0.5*np.cos(2*np.pi*( np.arange(-N/2,N/2)+np.remainder(N,2)/2 ) / N);
    return window

def calcCanonicalDualWindow(window,shiftLen):
    winLen = len(window)
    padLen = np.ceil(winLen/shiftLen)*shiftLen - winLen;
    dualWin = np.concatenate(( window, np.zeros(int(padLen)).astype(np.float64) ), axis=0)
    dualWin = np.reshape(dualWin,(int(len(dualWin)/shiftLen),int(shiftLen)))
    dualWin /= np.sum( np.abs(dualWin)**2, axis=0)
    return dualWin[:winLen].flatten()

def zeroPadForDGT(signal,shiftLen,fftLen):
    sigLen = len(signal);
    c = np.array(int(shiftLen) * int(fftLen) / math.gcd(int(shiftLen), int(fftLen))).astype(np.float64);
    sigLong = xp.hstack( (signal, xp.zeros( int(np.ceil(sigLen/c)*c-sigLen)) ) )    
    return sigLong

########################################################################################
# DNN savename 
def DNNfn( dnn_dir, DNNmode, in_dim, winLen, shiftLen, fftLen, winMode, lossMode, epoch, datetimeStr ): 
    return dnn_dir+DNNmode+'_' \
        +'stft'\
        +str(in_dim)+'ch'\
        +'_winL'+str(winLen)\
        +'_shift'+str(shiftLen)\
        +'_fftL'+str(fftLen)\
        +'_'+winMode\
        +'_'+lossMode\
        +'_epoch'+str(epoch).zfill(3)\
        +'_'+datetimeStr\
        +".h5"

#####################################################################
if __name__ == "__main__":
     print('Debug only')   
    


# Import Datasets
clean data : CMU Arctic Corpus
noise data : CMU Arctic Corpus + UrbanSound8K

In [46]:
print("...Loading...")

cleandata = np.load("clean_dataset.npy", allow_pickle=True)
noisedata = np.load("noise_dataset.npy", allow_pickle=True)

print("Convert numpy -> tensor")
datasets = []
for i in tqdm(range(len(cleandata))):
    _data = torch.from_numpy(cleandata[i].astype(np.float32)).clone()
    _label = torch.from_numpy(noisedata[i].astype(np.float32)).clone()
    data = (_data,_label)
    datasets.append(data)
    
print("Sprit Datasets to train/test")
# datasets = np.array(datasets)
train ,test = train_test_split(datasets,test_size = 0.2)
print("traindata size:", len(train),"/",len(datasets))
print("testdata size:",len(test),"/",len(datasets))

...Loading...
Convert numpy -> tensor


Sprit Datasets to train/test


In [58]:
print("...Loading...")

class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, transform = None):
        self.transform = transform

        self.data = np.load("clean_dataset.npy", allow_pickle=True)
        self.label = np.load("noise_dataset.npy", allow_pickle=True)

        self.datanum = len(self.data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label
    
dataset = Mydatasets()
print(len(dataset))

trainset ,testset = train_test_split(dataset,test_size = 0.2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batchSize, shuffle = True, num_workers = 2)
testloader = torch.utils.data.DataLoader(testset, batch_size = batchSize, shuffle = True, num_workers = 2)

...Loading...
15603
(array([0.00485229, 0.00567627, 0.00540161, ..., 0.        , 0.        ,
       0.        ], dtype=float32), array([-0.21405473, -0.39160603, -0.4057909 , ..., -0.19606587,
       -0.15675776, -0.09252358], dtype=float32))


In [59]:
trainset ,testset = train_test_split(dataset,test_size = 0.2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batchSize, shuffle = True, num_workers = 2)
testloader = torch.utils.data.DataLoader(testset, batch_size = batchSize, shuffle = True, num_workers = 2)

In [51]:
len(train)
len(datasets)

15603

In [7]:
fs = 16000

In [8]:
display(Audio(cleandata[0],rate=fs))
display(Audio(noisedata[0],rate=fs))

# Dataloader